In [1]:
library(tidyverse)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
marker_file_paths <- c("/home/jpm73279/genome_downloads/C4_markers/Os.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Pm.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Sb.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Uf.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Zm.c4_markers.bed")

# Function to read and process the file
process_file <- function(file_path) {
  species <- substr(basename(file_path), 1, 2)
  read_delim(file_path, delim = "\t", col_names = c("chrom", "start", "end", "geneID", "name", "type")) %>% 
    dplyr::select("chrom","start","end","geneID","name","type") %>%
    dplyr::mutate(species = tolower(species))
}

# Apply the function to each file and store results in a list
list_of_dataframes <- lapply(marker_file_paths, process_file)

# Optionally combine all dataframes into one if needed
markers <- bind_rows(list_of_dataframes) %>% 
    dplyr::select(species, name, type)

Rows: 33 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 62 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 31 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 28 Columns: 6

In [3]:
# zm_fimo_output <- read_tsv("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/zm_dir/all_acrs/zm_fimo_output/fimo.tsv")
# col_names <- c(
#   "chromosome_1", 
#   "start_1", 
#   "end_1", 
#   "gene_id", 
#   "gene_name", 
#   "chromosome_2", 
#   "start_2", 
#   "end_2", 
#   "acrs_id", 
#   "p_value"
# )
# zm_ms_intersect_file <- "/home/jpm73279/genome_downloads/C4_markers/Zm_ms_acrs_intersect.bed"
# zm_bs_intersect_file <- "/home/jpm73279/genome_downloads/C4_markers/Zm_bs_acrs_intersect.bed"
# zm_ms <- read_tsv(zm_ms_intersect_file, col_names = col_names) %>% 
#     mutate(type = "ms")

# zm_bs <- read_tsv(zm_bs_intersect_file, col_names = col_names) %>% 
#     mutate(type = "bs")

# combined_rows <- bind_rows(zm_ms, zm_bs) %>% 
#     dplyr::mutate(species = "zm")


# number_acrs_per_type <- combined_rows %>% 
#     dplyr::group_by(species, type) %>% 
#     dplyr::summarise(counts = n())


# zm_fimo_output <- zm_fimo_output %>% 
#     separate(sequence_name, c("acrID", "region"), sep = "::") %>% 
#     dplyr::filter(acrID %in% combined_rows$acrs_id)

# number_acrs_per_type <- combined_rows %>% 
#     dplyr::group_by(species, type) %>% 
#     dplyr::summarise(counts = n())

# intersection_combined <- combined_rows %>% 
#     dplyr::select(gene_id, gene_name, acrs_id, type, species)


# combined_fimo_intersect <- left_join(zm_fimo_output, intersection_combined, by = c("acrID" = "acrs_id"), relationship =  "many-to-many") %>% 
#     dplyr::select(motif_id, acrID, gene_id,gene_name,type,species) %>% 
#     unique()
#     #left_join(., number_acrs_per_type, by = c("type", "species"))

# contingency_start <- combined_fimo_intersect %>% 
#     dplyr::group_by(motif_id, type, species) %>% 
#     dplyr::summarise(count_motif_occurence = n()) %>% 
#     left_join(., number_acrs_per_type, by = c("type", "species")) %>% 
#     dplyr::rowwise() %>% 
#     mutate(non_motif_occurence = counts - count_motif_occurence)


In [4]:
generate_counts_contingency_table <- function(intersection_files, species_name) {
  col_names <- c(
    "chromosome_1", 
    "start_1", 
    "end_1", 
    "gene_id", 
    "gene_name", 
    "chromosome_2", 
    "start_2", 
    "end_2", 
    "acrs_id", 
    "p_value"
  )
  # Read the ms intersect file and mutate type
  zm_ms <- readr::read_tsv(intersection_files[[1]], col_names = col_names) %>% 
    dplyr::mutate(type = "bs")

  # Read the bs intersect file and mutate type
  zm_bs <- readr::read_tsv(intersection_files[[2]], col_names = col_names) %>% 
    dplyr::mutate(type = "ms")

  # Combine rows and mutate species
  combined_rows <- dplyr::bind_rows(zm_ms, zm_bs) %>% 
    dplyr::mutate(species = species_name)

  # Calculate number of acrs per type
  number_acrs_per_type <- combined_rows %>% 
    dplyr::group_by(species, type) %>% 
    dplyr::summarise(counts = n())
  
  return(number_acrs_per_type)
}



process_data <- function(fimo_output, intersection_files, species_name) {
  
  # Read the FIMO output
  zm_fimo_output <- read_tsv(fimo_output)
  
  col_names <- c(
    "chromosome_1", 
    "start_1", 
    "end_1", 
    "gene_id", 
    "gene_name", 
    "chromosome_2", 
    "start_2", 
    "end_2", 
    "acrs_id", 
    "p_value"
  )
  
  # Read the intersection files
  zm_ms <- read_tsv(intersection_files[[1]], col_names = col_names) %>% 
        mutate(type = "bs") %>% 
        dplyr::mutate(species = species_name)
  zm_bs <- read_tsv(intersection_files[[2]], col_names = col_names) %>% 
        mutate(type = "ms") %>% 
        dplyr::mutate(species = species_name)

        
  
  # Combine rows from both intersection files
  combined_rows <- bind_rows(zm_ms, zm_bs)
  
  # Process the FIMO output
  zm_fimo_filtered_output <- zm_fimo_output %>% 
    separate(sequence_name, c("acrID", "region"), sep = "::") %>% 
    dplyr::filter(acrID %in% combined_rows$acrs_id)
  
  # Extract selected columns from combined_rows
  intersection_combined <- combined_rows %>% 
    dplyr::select(gene_id, gene_name, acrs_id, type)
  
  # Combine the FIMO output with the intersection data
  combined_fimo_intersect <- left_join(zm_fimo_filtered_output, intersection_combined, by = c("acrID" = "acrs_id"), relationship = "many-to-many") %>% 
        dplyr::mutate(species = species_name)
    
    rm(zm_fimo_output)
      rm(zm_ms)
    rm(zm_bs)
    gc()
  return(combined_fimo_intersect)
}


process_data_null_files <- function(fimo_output, intersection_files, species_name) {
  
  # Read the FIMO output
  zm_fimo_output <- read_tsv(fimo_output)
  
  col_names <- c(
    "chromosome", 
    "start", 
    "end", 
    "acrs_id", 
    "p_value"
  )
  
  # Read the intersection files
  zm_ms <- read_tsv(intersection_files[[1]], col_names = col_names) %>% 
        mutate(type = "bs") %>% 
        mutate(gene_name = "null")
  zm_bs <- read_tsv(intersection_files[[2]], col_names = col_names) %>% 
        mutate(type = "ms") %>% 
        mutate(gene_name = "null")
        
  
  # Combine rows from both intersection files
  combined_rows <- bind_rows(zm_ms, zm_bs)
  
  # Process the FIMO output
  zm_fimo_filtered_output <- zm_fimo_output %>% 
    separate(sequence_name, c("acrID", "region"), sep = "::") %>% 
    dplyr::filter(acrID %in% combined_rows$acrs_id)
  
  # Extract selected columns from combined_rows
  intersection_combined <- combined_rows %>% 
    dplyr::select(gene_name, acrs_id, type)
  
  # Combine the FIMO output with the intersection data
  combined_fimo_intersect <- left_join(zm_fimo_filtered_output, intersection_combined, by = c("acrID" = "acrs_id"), relationship = "many-to-many") %>% 
        dplyr::mutate(species = species_name)
    
    rm(zm_fimo_output)
      rm(zm_ms)
    rm(zm_bs)
    gc()
  return(combined_fimo_intersect)
}


In [5]:
base_path <- "/home/jpm73279/genome_downloads/C4_markers/"
intersect_list <- list(
  Os = list(
    bs = paste0(base_path, "Os_bs_acrs_intersect.bed"),
    ms = paste0(base_path, "Os_ms_acrs_intersect.bed")
  ),
  Pm = list(
    bs = paste0(base_path, "Pm_bs_acrs_intersect.bed"),
    ms = paste0(base_path, "Pm_ms_acrs_intersect.bed")
  ),
  Sb = list(
    bs = paste0(base_path, "Sb_bs_acrs_intersect.bed"),
    ms = paste0(base_path, "Sb_ms_acrs_intersect.bed")
  ),
  Uf = list(
    bs = paste0(base_path, "Uf_bs_acrs_intersect.bed"),
    ms = paste0(base_path, "Uf_ms_acrs_intersect.bed")
  ),
  Zm = list(
    bs = paste0(base_path, "Zm_bs_acrs_intersect.bed"),
    ms = paste0(base_path, "Zm_ms_acrs_intersect.bed")
  )
)

In [7]:
zm_fimo_counts_all <- generate_counts_contingency_table(c(intersect_list$Zm$bs, intersect_list$Zm$ms), "zm")       
pm_fimo_counts_all <- generate_counts_contingency_table(c(intersect_list$Pm$bs, intersect_list$Pm$ms), "pm")       
os_fimo_counts_all <- generate_counts_contingency_table(c(intersect_list$Os$bs, intersect_list$Os$ms), "os")       
uf_fimo_counts_all <- generate_counts_contingency_table(c(intersect_list$Uf$bs, intersect_list$Uf$ms), "uf")       
sb_fimo_counts_all <- generate_counts_contingency_table(c(intersect_list$Sb$bs, intersect_list$Sb$ms), "sb")       

Rows: 13 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): chromosome_1, gene_id, gene_name, chromosome_2, acrs_id
dbl (5): start_1, end_1, start_2, end_2, p_value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 29 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): chromosome_1, gene_id, gene_name, chromosome_2, acrs_id
dbl (5): start_1, end_1, start_2, end_2, p_value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'species'. You can override using the
`.groups` argument.
Rows: 54 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): chrom

In [8]:
sb_fimo_counts_all

species,type,counts
<chr>,<chr>,<int>
sb,bs,14
sb,ms,28


In [9]:
all_species_fimo_counts <- bind_rows(zm_fimo_counts, pm_fimo_counts, os_fimo_counts,sb_fimo_counts,uf_fimo_counts)
all_species_total_counts <- bind_rows(zm_fimo_counts_all,pm_fimo_counts_all, os_fimo_counts_all, uf_fimo_counts_all, sb_fimo_counts_all)

In [10]:
all_species_fimo_counts

motif_id,motif_alt_id,acrID,region,start,stop,strand,score,p-value,q-value,matched_sequence,gene_id,gene_name,type,species
<chr>,<lgl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
HSF,NA,scACR_5926;mesophyll,chr1:226422323-226422824,450,463,-,21.5688,1.46e-08,0.06570,AGAAGGTTCTAGAA,Zm00001eb042970,PPT,ms,zm
BBRBPC,NA,scACR_42073;mesophyll,chr6:177097950-177098451,349,362,-,16.5854,9.56e-07,0.00282,AAGAGAGAGAAGGA,Zm00001eb296550,NAD-MDH2,ms,zm
MADS_MIKC,NA,scACR_1560;mesophyll,chr1:35695232-35695733,428,440,+,14.9458,1.22e-06,0.56700,CCCAAAATGGTAA,Zm00001eb011080,PEPCK,ms,zm
BBRBPC,NA,scACR_42073;mesophyll,chr6:177097950-177098451,353,366,-,16.1829,1.56e-06,0.00431,AAGAAAGAGAGAGA,Zm00001eb296550,NAD-MDH2,ms,zm
BBRBPC,NA,scACR_42073;mesophyll,chr6:177097950-177098451,347,360,-,16.1098,1.70e-06,0.00465,GAGAGAGAAGGAGA,Zm00001eb296550,NAD-MDH2,ms,zm
BES1,NA,scACR_51667;bundle_sheath,chr8:181465456-181465957,175,184,-,15.9024,2.10e-06,0.35600,TGCACGTGTG,Zm00001eb371140,NADP_ME,bs,zm
BBRBPC,NA,scACR_8033;mesophyll,chr1:289348515-289349016,237,250,-,15.3293,3.26e-06,0.00790,GGGAGAGAGGCAGA,Zm00001eb059070,NAD-MDH12,ms,zm
BBRBPC,NA,scACR_24813;mesophyll,chr3:220994969-220995470,364,377,-,15.0732,3.74e-06,0.00877,GCGAGAGAGCGCGA,Zm00001eb158800,CA2,ms,zm
BBRBPC,NA,scACR_24813;mesophyll,chr3:220994969-220995470,364,377,-,15.0732,3.74e-06,0.00877,GCGAGAGAGCGCGA,Zm00001eb158810,CA1,ms,zm


In [11]:
base_path <- "/home/jpm73279/genome_downloads/C4_markers/generate_null_ACR_files/"  # Adjust this to your directory
file_lists_null <- list(
  Os = list(
    bs = paste0(base_path, "Os_bs_acrs_intersect.null.bed..broad_acr_null_list.bed"),
    ms = paste0(base_path, "Os_ms_acrs_intersect.null.bed..broad_acr_null_list.bed")
  ),
  Pm = list(
    bs = paste0(base_path, "Pm_bs_acrs_intersect.null.bed..broad_acr_null_list.bed"),
    ms = paste0(base_path, "Pm_ms_acrs_intersect.null.bed..broad_acr_null_list.bed")
  ),
  Sb = list(
    bs = paste0(base_path, "Sb_bs_acrs_intersect.null.bed..broad_acr_null_list.bed"),
    ms = paste0(base_path, "Sb_ms_acrs_intersect.null.bed..broad_acr_null_list.bed")
  ),
  Uf = list(
    bs = paste0(base_path, "Uf_bs_acrs_intersect.null.bed..broad_acr_null_list.bed"),
    ms = paste0(base_path, "Uf_ms_acrs_intersect.null.bed..broad_acr_null_list.bed")
  ),
  Zm = list(
    bs = paste0(base_path, "Zm_bs_acrs_intersect.null.bed..broad_acr_null_list.bed"),
    ms = paste0(base_path, "Zm_ms_acrs_intersect.null.bed..broad_acr_null_list.bed")
  )
)

In [12]:
zm_fimo_counts_null <- process_data_null_files("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/zm_dir/all_acrs/zm_fimo_output/fimo.tsv",
                 c(file_lists_null$Zm$bs, file_lists_null$Zm$ms), "zm")       
pm_fimo_counts_null <- process_data_null_files("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/pm_dir/all_acrs/pm_fimo_output/fimo.tsv",
                 c(file_lists_null$Pm$bs, file_lists_null$Pm$ms), "pm")       
os_fimo_counts_null <- process_data_null_files("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/os_dir/all_acrs/os_fimo_output/fimo.tsv",
                 c(file_lists_null$Os$bs, file_lists_null$Os$ms), "os")       
uf_fimo_counts_null <- process_data_null_files("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/uf_dir/all_acrs/uf_fimo_output/fimo.tsv",
                 c(file_lists_null$Uf$bs, file_lists_null$Uf$ms), "uf")       
sb_fimo_counts_null <- process_data_null_files("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/06.motif_enrichment/sb_dir/all_acrs/sb_fimo_output/fimo.tsv",
                 c(file_lists_null$Sb$bs, file_lists_null$Sb$ms), "sb")       

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 149440 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): motif_id, sequence_name, strand, matched_sequence
dbl (5): start, stop, score, p-value, q-value
lgl (1): motif_alt_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 15 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): chromosome, acrs_id
dbl (2): start, end
lgl (1): p_value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 27 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
ch

In [13]:
fimo_counts_combined_null <- bind_rows(
  zm_fimo_counts_null,
  pm_fimo_counts_null,
  os_fimo_counts_null,
  uf_fimo_counts_null,
  sb_fimo_counts_null
)

In [14]:
head(fimo_counts_combined_null)

motif_id,motif_alt_id,acrID,region,start,stop,strand,score,p-value,q-value,matched_sequence,gene_name,type,species
<chr>,<lgl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
BBRBPC,NA,scACR_7004;broadly_accessible,chr1:261782646-261783147,266,279,-,21.9268,3.48e-09,1.93e-05,GAGAGAGAGAGAGA,null,ms,zm
BBRBPC,NA,scACR_7004;broadly_accessible,chr1:261782646-261783147,268,281,-,21.9268,3.48e-09,1.93e-05,GAGAGAGAGAGAGA,null,ms,zm
BBRBPC,NA,scACR_7004;broadly_accessible,chr1:261782646-261783147,264,277,-,19.8171,4.38e-08,1.93e-04,GAGAGAGAGAGAGT,null,ms,zm
BBRBPC,NA,scACR_5293;broadly_accessible,chr1:207077821-207078322,209,222,+,19.7683,5.50e-08,2.33e-04,GAGAGAGAGAGACA,null,ms,zm
BBRBPC,NA,scACR_5293;broadly_accessible,chr1:207077821-207078322,207,220,+,17.9756,2.96e-07,1.04e-03,AGGAGAGAGAGAGA,null,ms,zm
BBRBPC,NA,scACR_7004;broadly_accessible,chr1:261782646-261783147,270,283,-,17.8171,3.94e-07,1.33e-03,ACGAGAGAGAGAGA,null,ms,zm


# Analysis by Species and Cell Type

In [16]:
all_species_fimo_counts %>% 
    dplyr::select(motif_id, acrID, type,species) %>% 
    dplyr::group_by(motif_id, type, species) %>% 
    dplyr::summarise(count_motif_occurence = n()) %>% 
    left_join(., all_species_total_counts, by = c("type", "species")) %>% 
    dplyr::rowwise() %>% 
    mutate(non_motif_occurence = counts - count_motif_occurence) %>% 
    dplyr::filter(species == "sb")


`summarise()` has grouped output by 'motif_id', 'type'. You can override using
the `.groups` argument.


motif_id,type,species,count_motif_occurence,counts,non_motif_occurence
<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,bs,sb,1,14,13
AS2LOB,ms,sb,11,28,17
BBRBPC,bs,sb,12,14,2
BBRBPC,ms,sb,30,28,-2
CPP,bs,sb,1,14,13
E2FDP,bs,sb,2,14,12
E2FDP,ms,sb,4,28,24
GARP_ARR-B,ms,sb,2,28,26
GARP_G2-like,bs,sb,2,14,12


In [17]:
fimo_counts_combined_null %>% 
    dplyr::select(motif_id, acrID, type,species) %>% 
    dplyr::group_by(motif_id, type, species) %>% 
    dplyr::summarise(count_motif_occurence = n()) %>% 
    left_join(., all_species_total_counts, by = c("type", "species")) %>% 
    dplyr::rowwise() %>% 
    mutate(non_motif_occurence = counts - count_motif_occurence) %>% 
    dplyr::filter(species == "sb")


`summarise()` has grouped output by 'motif_id', 'type'. You can override using
the `.groups` argument.


motif_id,type,species,count_motif_occurence,counts,non_motif_occurence
<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,bs,sb,1,14,13
AS2LOB,ms,sb,16,28,12
BBRBPC,bs,sb,6,14,8
BBRBPC,ms,sb,17,28,11
BES1,bs,sb,1,14,13
BES1,ms,sb,4,28,24
CPP,bs,sb,3,14,11
CPP,ms,sb,5,28,23
E2FDP,ms,sb,2,28,26


In [18]:
all_species_fimo_counts %>% 
    dplyr::select(motif_id, type,species, gene_name) %>% 
    dplyr::group_by(motif_id, type, species,gene_name) %>% 
    dplyr::summarise(count_motif_occurence = n()) %>% 
    left_join(., all_species_total_counts, by = c("type", "species")) %>% 
    dplyr::rowwise() %>% 
    mutate(non_motif_occurence = counts - count_motif_occurence) %>% 
    dplyr::filter(gene_name %in% c("CA2", "CA1", "CA6", "CA_1__CA_2", "CA_2", "CA6_ortho_count_2")) %>% 
    arrange(motif_id, type, species)


`summarise()` has grouped output by 'motif_id', 'type', 'species'. You can
override using the `.groups` argument.


motif_id,type,species,gene_name,count_motif_occurence,counts,non_motif_occurence
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,ms,os,CA_1__CA_2,1,25,24
AS2LOB,ms,sb,CA1,3,28,25
AS2LOB,ms,sb,CA6_ortho_count_2,2,28,26
AS2LOB,ms,zm,CA1,2,29,27
AS2LOB,ms,zm,CA2,2,29,27
AS2LOB,ms,zm,CA6,3,29,26
BBRBPC,ms,sb,CA1,10,28,18
BBRBPC,ms,sb,CA6_ortho_count_2,12,28,16
BBRBPC,ms,zm,CA1,1,29,28


In [19]:
unique(all_species_fimo_counts$gene_name)

[1] "PPT"                                                                                   
 [2] "NAD-MDH2"                                                                              
 [3] "PEPCK"                                                                                 
 [4] "NADP_ME"                                                                               
 [5] "NAD-MDH12"                                                                             
 [6] "CA2"                                                                                   
 [7] "CA1"                                                                                   
 [8] "CA6"                                                                                   
 [9] "RUBISCO_activase1"                                                                     
[10] "RBCA2"                                                                                 
[11] "RBCA1"                                                                                 
[12] "PPDK"                                                                                  
[13] "NADP_ME1"                                                                              
[14] "PEPC1"                                                                                 
[15] "AlaAT3_ortho_count_2__AlaAT4_ortho_count_2"                                            
[16] "NAD-ME_2"                                                                              
[17] "NAD-ME_1"                                                                              
[18] "CA_1"                                                                                  
[19] "PPDK_1"                                                                                
[20] "RUBISCO_activase1_ortho_count_1"                                                       
[21] "NADP-ME_1"                                                                             
[22] "AlaAT2_ortho_count_1"                                                                  
[23] "AlaAT2_ortho_count_2"                                                                  
[24] "PPT_4"                                                                                 
[25] "PEP-CK_3"                                                                              
[26] "NADP-MDH_1"                                                                            
[27] "PPDK-RP"                                                                               
[28] "AlaAT1_ortho_count_1"                                                                  
[29] "AlaAT3"                                                                                
[30] "NADP-ME_2"                                                                             
[31] "PPT_3"                                                                                 
[32] "NAD-MDH1"                                                                              
[33] "CA_2"                                                                                  
[34] "CA6_ortho_count_2"                                                                     
[35] "RBCA2_ortho_count_7__RBCA1_ortho_count_7"                                              
[36] "BASS2"                                                                                 
[37] "PEPC_2"                                                                                
[38] "RBCA2_ortho_count_4__RBCA1_ortho_count_4"                                              
[39] "AspAT_1"                                                                               
[40] "PPDK_2"                                                                                
[41] "RBCA2_ortho_count_1__RBCA1_ortho_count_1"                                              
[42] "RBCA2_ortho_count_2__RBCA1_ortho_count_2"                                              
[43] "RBCA2_ortho_count_3__RBCA1_ortho_count_3"      

In [20]:
all_species_fimo_counts %>% 
    dplyr::select(motif_id, type,species, gene_name) %>% 
    dplyr::group_by(motif_id, type, species,gene_name) %>% 
    dplyr::summarise(count_motif_occurence = n()) %>% 
    left_join(., all_species_total_counts, by = c("type", "species")) %>% 
    dplyr::rowwise() %>% 
    mutate(non_motif_occurence = counts - count_motif_occurence) %>% 
    dplyr::filter(gene_name %in% c("RBCA2_ortho_count_4__RBCA1_ortho_count_4", "RBCA2_ortho_count_2__RBCA1_ortho_count_2-CK_2",
                                  "RUBISCO_activase1", "RBCA2", "RBCA1")) %>% 
    arrange(motif_id, type, species)


`summarise()` has grouped output by 'motif_id', 'type', 'species'. You can
override using the `.groups` argument.


motif_id,type,species,gene_name,count_motif_occurence,counts,non_motif_occurence
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,bs,pm,RBCA2_ortho_count_4__RBCA1_ortho_count_4,2,54,52
AS2LOB,ms,zm,RBCA1,1,29,28
BBRBPC,bs,uf,RUBISCO_activase1,1,35,34
BBRBPC,ms,zm,RBCA1,2,29,27
BBRBPC,ms,zm,RBCA2,4,29,25
CPP,bs,pm,RBCA2_ortho_count_4__RBCA1_ortho_count_4,1,54,53
CPP,bs,uf,RUBISCO_activase1,1,35,34
GARP_G2-like,bs,pm,RBCA2_ortho_count_4__RBCA1_ortho_count_4,1,54,53
HD-Zip_IV,bs,sb,RUBISCO_activase1,1,14,13


In [21]:
collpsed_by_species_cell_type <- all_species_fimo_counts %>% 
    dplyr::select(motif_id, acrID, type,species) %>% 
    unique() %>% 
    dplyr::group_by(motif_id, type, species) %>% 
    dplyr::summarise(count_motif_occurence = n()) %>% 
    left_join(., all_species_total_counts, by = c("type", "species")) %>% 
    dplyr::rowwise() %>% 
    mutate(non_motif_occurence = counts - count_motif_occurence)


complete_combination <- expand.grid(
  motif_id = unique(fimo_counts_combined_null$motif_id),
  type = unique(fimo_counts_combined_null$type),
  species = unique(fimo_counts_combined_null$species)
)

null_collapsed_species_cell_type <- fimo_counts_combined_null %>%
  dplyr::select(motif_id, acrID, type, species) %>% 
  unique() %>% 
  dplyr::group_by(motif_id, type, species) %>% 
  dplyr::summarise(null_count_motif_occurence = n(), .groups = 'drop') %>% 
  # left join with complete combination to ensure every possible instance exists
  right_join(complete_combination, by = c("motif_id", "type", "species")) %>% 
  replace_na(list(null_count_motif_occurence = 0)) %>% # replace NA with 0
  left_join(., all_species_total_counts, by = c("type", "species")) %>% 
  dplyr::rowwise() %>% 
  mutate(null_count_non_motif_occurence = counts - null_count_motif_occurence)

`summarise()` has grouped output by 'motif_id', 'type'. You can override using
the `.groups` argument.


In [22]:
combined_null_real <- left_join(collpsed_by_species_cell_type, null_collapsed_species_cell_type,by = join_by(motif_id, type, species, counts)) %>% 
    dplyr::select(-counts) 

In [23]:
fisher_test_from_row <- function(count_motif_occurence, non_motif_occurence, null_count_motif_occurence, null_count_non_motif_occurence) {
  matrix_data <- c(count_motif_occurence, null_count_motif_occurence, non_motif_occurence, null_count_non_motif_occurence)
  matrix_for_test <- matrix(matrix_data, nrow = 2)
  fisher_result <- fisher.test(matrix_for_test)
  return(fisher_result$p.value)
}

# Apply the function to each row
combined_null_real.pvals <- combined_null_real %>%
  rowwise() %>%
  mutate(p_value = purrr::pmap_dbl(list(count_motif_occurence, non_motif_occurence, null_count_motif_occurence, null_count_non_motif_occurence), fisher_test_from_row))

print(combined_null_real.pvals)

# A tibble: 115 × 8
# Rowwise: 
   motif_id type  species count_motif_occurence non_motif_occurence
   <chr>    <chr> <chr>                   <int>               <int>
 1 AS2LOB   bs    os                          3                  11
 2 AS2LOB   bs    pm                          7                  47
 3 AS2LOB   bs    sb                          1                  13
 4 AS2LOB   bs    uf                          9                  26
 5 AS2LOB   bs    zm                          1                  12
 6 AS2LOB   ms    os                          9                  16
 7 AS2LOB   ms    pm                          6                  32
 8 AS2LOB   ms    sb                          6                  22
 9 AS2LOB   ms    uf                          8                  25
10 AS2LOB   ms    zm                          5                  24
# ℹ 105 more rows
# ℹ 3 more variables: null_count_motif_occurence <int>,
#   null_count_non_motif_occurence <int>, p_value <dbl>


In [24]:
combined_null_real.pvals %>% 
    dplyr::filter(p_value < .05)

motif_id,type,species,count_motif_occurence,non_motif_occurence,null_count_motif_occurence,null_count_non_motif_occurence,p_value
<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>
AS2LOB,ms,pm,6,32,16,22,0.02164525
BBRBPC,bs,pm,8,46,19,35,0.02518857
GARP_G2-like,bs,pm,8,46,1,53,0.03144815


In [25]:
head(collpsed_by_species_cell_type)
head(null_collapsed_species_cell_type)

motif_id,type,species,count_motif_occurence,counts,non_motif_occurence
<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,bs,os,3,14,11
AS2LOB,bs,pm,7,54,47
AS2LOB,bs,sb,1,14,13
AS2LOB,bs,uf,9,35,26
AS2LOB,bs,zm,1,13,12
AS2LOB,ms,os,9,25,16


motif_id,type,species,null_count_motif_occurence,counts,null_count_non_motif_occurence
<chr>,<chr>,<chr>,<int>,<int>,<int>
AS2LOB,bs,os,2,14,12
AS2LOB,bs,pm,11,54,43
AS2LOB,bs,sb,1,14,13
AS2LOB,bs,uf,7,35,28
AS2LOB,bs,zm,3,13,10
AS2LOB,ms,os,9,25,16


## Look into specific genes


In [34]:
just_counts <- all_species_fimo_counts %>% 
    group_by(species, type, gene_name,motif_id) %>% 
    dplyr::summarise(motif_count = n()) %>% 
    dplyr::arrange(desc(motif_count))
    

`summarise()` has grouped output by 'species', 'type', 'gene_name'. You can
override using the `.groups` argument.


In [39]:
just_counts %>% 
    dplyr::filter(gene_name == "PPDK")

species,type,gene_name,motif_id,motif_count
<chr>,<chr>,<chr>,<chr>,<int>
uf,ms,PPDK,E2FDP,4
sb,ms,PPDK,BBRBPC,3
uf,ms,PPDK,BBRBPC,2
sb,ms,PPDK,E2FDP,1
uf,bs,PPDK,AS2LOB,1
uf,ms,PPDK,AS2LOB,1
uf,ms,PPDK,CPP,1
uf,ms,PPDK,GARP_ARR-B,1
uf,ms,PPDK,GARP_G2-like,1
